In [7]:
import pandas as pd
import torch
import matplotlib.pyplot as plt
from transformers import BertTokenizer, BertForSequenceClassification

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [8]:
df = pd.read_csv('data/mytrain.csv')
df = df.iloc[:, -2:]
df

,text,generated
0,Candidate what do you think of when I say that...,0
1,"Ever since it was first invented, the automobi...",0
2,Have you ever come back to school after a long...,0
3,"Imagine you receiving advice from someone, but...",0
4,If you waych the news chaces are you probelly ...,0
...,...,...
3995,"Introduction, I planned my paper before writi...",1
3996,"[Your Name]\n[Your Address]\n[City, State, ZIP...",1
3997,I believe that students would greatly benefit ...,1
3998,Standardized testing has been around for a lon...,1


In [11]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['generated'], test_size=0.2, random_state=42)
train_data = list(zip(X_train, y_train))
test_data = list(zip(X_test, y_test))

In [12]:
# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Tokenize and preprocess the data
def preprocess_data(texts, labels):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")
    print(inputs)
    input_ids = inputs["input_ids"].squeeze()
    attention_mask = inputs["attention_mask"].squeeze()
    return input_ids, attention_mask, labels

# Preprocess the training and testing data
train_dataset = torch.utils.data.TensorDataset(*[torch.tensor(t) for t in preprocess_data(*zip(*train_data))])
test_dataset = torch.utils.data.TensorDataset(*[torch.tensor(t) for t in preprocess_data(*zip(*test_data))])

# Compile and fit the model
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss()

epochs = 2
for epoch in range(epochs):
    model.train()
    for input_ids, attention_mask, labels in train_dataset:
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids.unsqueeze(0), attention_mask=attention_mask.unsqueeze(0), labels=labels.unsqueeze(0))
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_labels = []
    val_preds = []
    with torch.no_grad():
        for input_ids, attention_mask, labels in test_dataset:
            outputs = model(input_ids=input_ids.unsqueeze(0), attention_mask=attention_mask.unsqueeze(0))
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).item()  # Use item() for scalar tensor
            val_labels.append(labels.item())  # Use item() for scalar tensor
            val_preds.append(preds)


    val_accuracy = accuracy_score(val_labels, val_preds)
    print(f"Epoch {epoch + 1}/{epochs}, Validation Accuracy: {val_accuracy:.4f}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

{'input_ids': tensor([[  101,  2043, 20059,  ...,  1996,  2783,   102],
        [  101,  5702, 11691,  ...,  4304,  3574,   102],
        [  101,  4931,  1010,  ...,  2064,  2130,   102],
        ...,
        [  101,  2296,  2711,  ...,  2111,  1999,   102],
        [  101,  1996, 14686,  ..., 11618,  2256,   102],
        [  101,  2516,  1024,  ...,  2031,  2195,   102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}


C:\Users\Steven\AppData\Local\Temp\ipykernel_39796\3313133439.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = torch.utils.data.TensorDataset(*[torch.tensor(t) for t in preprocess_data(*zip(*train_data))])


KeyboardInterrupt: 